In [59]:
import requests
import re
import warnings
import urllib.request

from webbot import Browser
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [60]:
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [61]:
common_words_list = pd.read_csv("Common_Articles_List.csv")
personality = pd.read_csv("Personality.csv", sep=',')
software = pd.read_csv("Programming_Languages.csv")

common = common_words_list["word"].to_list()
personality = [x.lower() for x in personality["Traits"]]
software = [x.lower() for x in software["languages"]]

In [4]:
def get_indeed_page_numbers(page):
    clock = False
    page_numbers_element = page.find_all("span", class_="pn", text=True)

    #First Loop To Give Us the 1st 5 Pages if available
    page_number_list = []
    for items in page_numbers_element:
        #print(items)
        page_number_list.append("https://www.indeed.com" + items.parent["href"])
    
    if len(page_numbers_element) == 4:
        
        loop_counter = 5
        
        while clock is False:
            last_page = page_number_list[-1]
            analysis_page = requests.get(last_page)
            soup = BeautifulSoup(analysis_page.content, "html.parser")
            new_link_list = soup.find_all("span", class_="pn", text=True)
            
            additional_counter = int(new_link_list[-1].text) - loop_counter
            
            if additional_counter < 1: 
                clock = True
                
            else: 
                for items in new_link_list[(additional_counter * -1):]:
                    loop_counter = int(items.text)
                    page_number_list.append("https://www.indeed.com" + items.parent["href"])
                clock = False
        
        
    return page_number_list

In [5]:
options = Options()
options.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome(ChromeDriverManager().install())

#User Input and Driver Link
driver.get('https://indeed.com/')
job_title = driver.find_element_by_id('text-input-what')
job_title.send_keys("Data Analyst")
location = driver.find_element_by_id("text-input-where")
location.send_keys("test")
locate_clear_field = driver.find_element_by_class_name("icl-TextInputClearable-icon")
locate_clear_field.click()
fixed_location = location.send_keys("Washington, DC")
submit_button = driver.find_element_by_class_name("yosegi-InlineWhatWhere-primaryButton")
submit_button.send_keys(Keys.RETURN)


current_page_url = driver.current_url
indeed_url = "https://www.indeed.com"

analysis_page = requests.get(current_page_url)
soup = BeautifulSoup(analysis_page.content, "html.parser")


additional_page_links = get_indeed_page_numbers(soup)

print(len(additional_page_links), " Potential Jobs Pages")



    
job_links = []    
job_listings = soup.find_all('a', class_="tapItem")
counter = 0
for job in job_listings:
    job_links.append(indeed_url + job["href"])
    

for page_link in additional_page_links:
    web_page = requests.get(page_link)
    soup = BeautifulSoup(web_page.content, "html.parser")
    job_listings = soup.find_all('a', class_="tapItem")
  
    
    for job in job_listings:
        job_links.append(indeed_url + job["href"])

        
print("Total Jobs Analyzed: ", len(job_links))



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Julio\.wdm\drivers\chromedriver\win32\100.0.4896.60]


62  Potential Jobs Pages
Total Jobs Analyzed:  750


In [12]:
counter = 0
page_body_text = []

for job_page in job_links:
    loaded_page = requests.get(job_page)
    soup = BeautifulSoup(loaded_page.text, "html.parser")
        
    
    page_body = soup.find_all("div", class_="jobsearch-JobComponent-description")
        
    for elements in page_body:
        text = elements.get_text()
        
        page_body_text.append(text)
        counter += 1
        
print(len(job_links))
print("Total Job Descriptions Analyzed: ", len(page_body_text))
print("DONE")

750
Total Job Descriptions Analyzed:  597
DONE


In [16]:
counter = 0
unwanted_chars = [" ", ",", "/", "'\'", ".", "(", ")", ";", '"', "'", ":", "-", "_", 
                  "&", "[", "]", "*", "?", "#", "!", "%", "®", "“", '”', ":", "~"]

additional_splitting = ["/", "(", ")", "%", "/", "."]
final_list = []
word_frequency = {}

for items in page_body_text:
    text = re.sub("\n", " ", items)
    clean_list = text.split()
    
    for word in clean_list:
        split_words = []
        if word != "":
                
            if len(word) > 1:
                
                cleaned = False
                #print("Word Going Into Machine: ", word)
                while cleaned is False:
                    
                    if len(word) > 1 and word[len(word)-1] in unwanted_chars:
                        word = word[:-1]
                
                        if len(word) > 1 and word[0] in unwanted_chars:
                            word = word[1:]
                    
                    
                    if len(word) > 1 and word[0] in unwanted_chars:
                        word = word[1:]
                
                        if len(word) > 1 and word[len(word)-1] in unwanted_chars:
                            word = word[:-1]
                    
                    
                    if len(word) == 1:
                        if word in unwanted_chars:
                            #print("Word Deleted: ", word)
                            cleaned = True
                        
                        else:
                            #print("Passed WORD = 1: ", word)
                            #print("   ")
                            final_list.append(word.lower().strip())
                            cleaned = True
                        
                    elif word[0] not in unwanted_chars and word[len(word) - 1] not in unwanted_chars:
                        #print("Passed ELIF: ", word)
                        #print("   ")
                        if word[-2:] == "'s" or word[-2:] == "’s":
                            #print("Machine Processing S Condition: ", word)
                            word = word[:-2] + "s"
                            #print("Passed Condition: ", word)
                            #print("   ")
            
                        if "$" in word:
                            #print("Machine Processing Money Condition: ", word)
                            word = "$" + word.split("$", 1)[1]
                            #print("Passed Condition: ", word)
                            #print("   ")
                        
                        for symbols in additional_splitting:
                            if symbols in word: 
                                split_words = word.split(symbols)
                                #print("Machine Processing Extra Symbols in Word Condition: ", word)
                                #print("Passed Condition: ", split_words)
                                #print("   ")
                        
                        if len(split_words) == 0: 
                            final_list.append(word.lower().strip())
                            cleaned = True
                        else: 
                            for words in split_words:
                                #print("Word Added Based On Extra Symbol Condition: ", words)
                                #print("     ")
                                final_list.append(words.lower().strip())
                                cleaned = True
                    
                    else: 
                        cleaned = False
                        
            else: 
                if word not in unwanted_chars:
                    #print("Passed SINGLE LETTER NOT IN CHARS: ", word)
                    #print("   ")
                    final_list.append(word.lower().strip())
                
            
        
print("Done")
print(len(final_list))

Done
358465


In [33]:
 for words in final_list:
        if words not in word_frequency:
            word_frequency[words] = 0
        else:
            word_frequency[words] += 1

len(word_frequency)

12233

In [63]:
for words in common: 
    if words in word_frequency.keys():
        del word_frequency[words]
    
sorted_word_frequency_list = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)


In [96]:
programming_languages = []
personality_traits = []
salary_expectations = []

for items in sorted_word_frequency_list: 
    if items[0] in software:
        programming_languages.append(items)
    
    elif items[0] in personality:
        personality_traits.append(items)
        
    elif "$" in items[0]:
        if len(items[0]) == 3:
            money = items[0]
            points = items[1]
            money = money
            salary_expectations.append((money, points))
            continue
        salary_expectations.append(items)

ValueError: invalid literal for int() with base 10: '3b'

In [91]:
programming_languages

[('sql', 1676),
 ('python', 904),
 ('r', 692),
 ('sas', 432),
 ('aws', 320),
 ('c', 244),
 ('oracle', 208),
 ('java', 100),
 ('mysql', 72),
 ('javascript', 68),
 ('css', 64),
 ('xml', 60),
 ('postgresql', 60),
 ('html', 44),
 ('matlab', 32),
 ('mongodb', 32),
 ('bash', 28),
 ('c++', 20),
 ('ada', 16),
 ('ruby', 12),
 ('db2', 12),
 ('elasticsearch', 8),
 ('redis', 4),
 ('sqlite', 4),
 ('scala', 4)]

In [92]:
personality_traits

[('knowledge', 2824),
 ('complex', 1376),
 ('understanding', 1048),
 ('responsible', 912),
 ('innovative', 736),
 ('critical', 692),
 ('physical', 600),
 ('flexible', 496),
 ('competitive', 456),
 ('active', 392),
 ('religious', 392),
 ('creative', 336),
 ('firm', 332),
 ('focused', 320),
 ('dynamic', 320),
 ('criminal', 284),
 ('deep', 268),
 ('dedicated', 252),
 ('passionate', 252),
 ('formal', 216),
 ('political', 196),
 ('challenging', 192),
 ('driving', 188),
 ('solid', 180),
 ('efficient', 176),
 ('confidential', 176),
 ('independent', 152),
 ('exciting', 140),
 ('sharing', 140),
 ('logical', 128),
 ('secure', 128),
 ('sensitive', 124),
 ('organized', 116),
 ('patient', 104),
 ('capable', 100),
 ('generous', 100),
 ('determined', 96),
 ('artificial', 96),
 ('practical', 96),
 ('objective', 92),
 ('progressive', 88),
 ('articulate', 88),
 ('adaptable', 84),
 ('curious', 80),
 ('dependent', 76),
 ('healthy', 76),
 ('thorough', 72),
 ('directed', 72),
 ('moderate', 68),
 ('reliable'

In [93]:
salary_expectations

[('4545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454545454